In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
import lightgbm as lgb
import xgboost as xgb
from functools import partial
import copy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from hyperopt import hp, tpe, Trials, fmin, space_eval
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv("../../../20200125atma/input/train.csv")
test = pd.read_csv("../../../20200125atma/input/test.csv")
userlog = pd.read_csv("../../../20200125atma/input/user_log.csv")
poi = pd.read_csv("../../../20200125atma/input/poi.csv")

In [84]:
import cv2

list_y = []
for i in range(13411):
    img = cv2.imread('images/'+str(i)+'.png')
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # HSV 色空間に変換

    red = cv2.inRange(hsv, np.array([145, 70, 0]), np.array([180, 255, 255]))
    yellow = cv2.inRange(hsv, (15,0,0), (36, 255, 255))
    green = cv2.inRange(hsv, np.array([30, 190, 0]), np.array([90, 255, 255]))
    blue = cv2.inRange(hsv, np.array([108, 121, 0]), np.array([120, 255, 255]))
    white = cv2.inRange(hsv, np.array([108, 21, 0]), np.array([255, 70, 255]))

    # 白だけゴミがあるので、収縮演算
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    white = cv2.erode(yellow, kernel)

    bin_imgs = {'red': red, 'yellow': yellow, 'green': green,
            'blue': blue, 'white': white}

    for label, bin_img in bin_imgs.items():
        contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = list(filter(lambda cnt: len(cnt) > 30, contours))
        count = len(contours)
    
        if label == "yellow":
            list_y.append(count)

In [85]:
def preprocess(train, test, userlog, poi):
    userlog["sysname"][userlog.sysname == "ANDROID"] = "Android"
    userlog["lang"][userlog.lang == "ja_JP"] = "ja_JP"
    poi = poi.rename(columns={"latitude": "store_lat", "longitude": "store_lon"})
    train = pd.merge(train, poi, on ="pid", how = "left")
    test = pd.merge(test, poi, on ="pid", how = "left")
    userlog = pd.merge(userlog, train[["session_id", "store_lat", "store_lon", "radius"]], on="session_id", how="left")
    userlog = pd.merge(userlog, test[["session_id", "store_lat", "store_lon", "radius"]], on="session_id", how="left")
    userlog["store_lat"] = np.nanmax(userlog[["store_lat_x", "store_lat_y"]], axis=1)
    userlog["store_lon"] = np.nanmax(userlog[["store_lon_x", "store_lon_y"]], axis=1)
    userlog["radius"] = np.nanmax(userlog[["radius_x", "radius_y"]], axis=1)
    drop_features = ["store_lat_x", "store_lat_y", "store_lon_x", "store_lon_y", "radius_x", "radius_y"]
    userlog.drop(drop_features, axis=1, inplace=True)
    userlog["distance"] = np.sqrt((userlog["latitude"]- userlog["store_lat"])**2 + (userlog["longitude"]-userlog["store_lon"])** 2 )
    userlog["time"] = userlog["hour"].map(str) + str(":") + userlog["minute"].map(str) + str(":") + userlog["second"].map(str)
    userlog["time"] = pd.to_datetime(userlog['time'],format= '%H:%M:%S' )
    userlog = userlog.sort_values(["session_id", "hour", "minute", "second"]).reset_index(drop=True)
    userlog["virtual_dis"] = 6370 * np.arccos(np.sin(userlog["latitude"])*np.sin(userlog["store_lat"]) + np.cos(userlog["latitude"])*np.cos(userlog["store_lat"])*np.cos(userlog["longitude"]-userlog["store_lon"]))
    userlog["in_store"] = userlog["virtual_dis"] < userlog["radius"]
    #userlog["JP"] = userlog["lang"].apply(lambda x: 1 if "ja" in x or "JP" in x else 0)
    #tmp = userlog.groupby("session_id").agg(["std"])[["latitude", "longitude", "categorical_1", 
    #                                               "categorical_2", "categorical_3", "categorical_4", "categorical_5", "categorical_6"]].reset_index(drop=False)
    #userlog = pd.merge(userlog, tmp, on="session_id", how="left")
    return train, test, userlog, poi
train, test, userlog, poi = preprocess(train, test, userlog, poi)

In [ ]:
def encode(train, test, userlog):
    os_list =  sorted(list(set(userlog['sysname'].unique())))
    os_map = dict(zip(os_list, np.arange(len(os_list))))
    userlog["sysname"] = userlog["sysname"].map(os_map)
    
    lang_list =  sorted(list(set(userlog['lang'].unique())))
    lang_map = dict(zip(lang_list, np.arange(len(lang_list))))
    userlog["lang"] = userlog["lang"].map(lang_map)
    
    timezone_list =  sorted(list(set(userlog['timezone'].unique())))
    timezone_map = dict(zip(timezone_list, np.arange(len(timezone_list))))
    userlog["timezone"] = userlog["timezone"].map(timezone_map)
    
    #cat1_list =  sorted(list(set(userlog['categorical_1'].unique())))
    #cat2_list =  sorted(list(set(userlog['categorical_2'].unique())))
    #cat3_list =  sorted(list(set(userlog['categorical_3'].unique())))
    #cat4_list =  sorted(list(set(userlog['categorical_4'].unique())))
    #cat5_list =  sorted(list(set(userlog['categorical_5'].unique())))
    #cat6_list =  sorted(list(set(userlog['categorical_6'].unique())))

    return train, test, userlog
train, test, userlog = encode(train, test, userlog)

In [ ]:
userlog.head(3)

,latitude,longitude,sysname,optout,lang,timezone,session_id,hour,minute,second,day_of_week,categorical_1,categorical_2,categorical_3,categorical_4,categorical_5,categorical_6,store_lat,store_lon,radius,distance,time,virtual_dis,in_store
0,35.650300,46.780509,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,0,33,31,6,4,0,44,1.0,1,187,35.651739,46.776134,118.0,0.004606,1900-01-01 00:33:31,15.744307,True
1,35.650298,46.780512,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,0,33,31,6,4,1,44,1.0,1,187,35.651739,46.776134,118.0,0.004609,1900-01-01 00:33:31,15.758882,True
2,35.650286,46.780524,0,0,9,1,0003f26df5d8b928b416fba58efd5c91,2,18,33,6,4,2,44,1.0,1,187,35.651739,46.776134,118.0,0.004624,1900-01-01 02:18:33,15.832117,True


In [ ]:
def get_logdata(user_sample):
    day_of_week_counts = {"day"+str(day) : 0 for day in range(7)}
    #cat1_counts = {"cat1_"+str(ele) : 0 for ele in cat1_list}
    #cat2_counts = {"cat2_"+str(ele) : 0 for ele in cat2_list}
    #cat3_counts = {"cat3_"+str(ele) : 0 for ele in cat3_list}
    #cat4_counts = {"cat4_"+str(ele) : 0 for ele in cat4_list}
    #cat5_counts = {"cat5_"+str(ele) : 0 for ele in cat5_list}
    #cat6_counts = {"cat6_"+str(ele) : 0 for ele in cat6_list}

    all_results = []
    features = {"accumulated_actions":0}
    features["accumulated_actions"] = user_sample.shape[0]
    features["session_id"] = user_sample.iloc[0]["session_id"]
    features["OS"] = user_sample.iloc[0]["sysname"]
    features["lang"] = user_sample.iloc[0]["lang"]
    features["timezone"] = user_sample.iloc[0]["timezone"]
    features["optout_count"] = np.sum(user_sample["optout"])
    distance = np.array(user_sample["distance"])
    features["max_dist"] = np.nanmax(distance)
    features["min_dist"] = np.nanmin(distance)
    features["std_dist"] = np.nanstd(distance)
    #features["mean_dist"] = np.nanmean(distance)
    #features["sum_dist"] = np.nanmean(distance)
    #features["max_lat"] = np.max(user_sample["latitude"])
    #features["min_lat"] = np.min(user_sample["latitude"])
    #features["max_lon"] = np.max(user_sample["longitude"])
    #features["min_lon"] = np.min(user_sample["longitude"])
    #features["range"] = (features["max_lat"] - features["min_lat"]) * (features["max_lon"]- features["min_lon"])
    #features["JP"] = user_sample.iloc[0]["JP"]
    
    n_of_days = Counter(user_sample['day_of_week']) 
    for key in n_of_days.keys():
        day_of_week_counts["day"+str(key)] += n_of_days[key]
    features.update(day_of_week_counts)
    
    #vir_distance = np.array(user_sample["virtual_dis"])
    #features["max_vir"] = np.nanmax(vir_distance)
    #features["min_vir"] = np.nanmin(vir_distance)
    #features["std_vir"] = np.nanstd(vir_distance)
    #features["mean_vir"] = np.nanmean(vir_distance)
    features["in_store"] = np.sum(user_sample["in_store"])
    #features["cat4"] = np.sum(user_sample["cat4"])
    
    #features["mean_lat"] = user_sample.iloc[0]["'latitude', 'mean'"]
    #features["std_lat"] = user_sample.iloc[0]["latitude"]["std"]
    #features["mean_lon"] = user_sample.iloc[0]["longitude"]["mean"]
    #features["std_lon"] = user_sample.iloc[0]["longitude"]["mean"]
    #feautres["mean_cat5"] = user_sample.iloc[0]["categorical_5"]["mean"]
    #features["std_cat5"] = user_sample.iloc[0]["latitude"]["mean"]["categorical_6"]["std"]
    #features["mean_cat6"] = user_sample.iloc[0]["categorical_5"]["mean"]
    #features["std_cat6"] = user_sample.iloc[0]["categorical_6"]["std"]
    
    #diff = np.array(user_sample['time'].diff(1) / np.timedelta64(1,'s'))
    #features["max_diff"] = np.nanmax(diff)
    #features["min_diff"] = np.nanmin(diff)
    #features["std_diff"] = np.nanstd(diff)
    #features["mean_diff"] = np.nanmean(diff)
    
    #features["catogorical_1_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_1"])]["categorical_1"]))
    #features["catogorical_2_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_2"])]["categorical_2"]))
    #features["catogorical_3_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_3"])]["categorical_3"]))
    #features["catogorical_4_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_4"])]["categorical_4"]))
    #features["catogorical_5_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_5"])]["categorical_5"]))
    #features["catogorical_6_kind"] = len(np.unique(user_sample[~np.isnan(user_sample["categorical_6"])]["categorical_6"]))
    
    #n_of_cat = Counter(user_sample['categorical_1']) 
    #for key in n_of_cat.keys():
    #    cat1_counts["cat1_"+str(key)] += n_of_cat[key]
    #features.update(cat1_counts)
    #n_of_cat = Counter(user_sample['categorical_2']) 
    #for key in n_of_cat.keys():
    #    cat2_counts["cat2_"+str(key)] += n_of_cat[key]
    #features.update(cat2_counts)
    #n_of_cat = Counter(user_sample['categorical_3']) 
    #for key in n_of_cat.keys():
    #    cat3_counts["cat3_"+str(key)] += n_of_cat[key]
    #features.update(cat3_counts)
    #n_of_cat = Counter(user_sample['categorical_4']) 
    #for key in n_of_cat.keys():
    #    cat4_counts["cat4_"+str(key)] += n_of_cat[key]
    #features.update(cat4_counts)
    #n_of_cat = Counter(user_sample['categorical_5']) 
    #for key in n_of_cat.keys():
    #    cat5_counts["cat5_"+str(key)] += n_of_cat[key]
    #features.update(cat5_counts)
    #n_of_cat = Counter(user_sample['categorical_6']) 
    #for key in n_of_cat.keys():
    #    cat6_counts["cat6_"+str(key)] += n_of_cat[key]
    #features.update(cat6_counts)
    
    all_results.append(features)
    return all_results

In [ ]:
def get_log_info(userlog):
    compiled_log = []

    for i, (ses_id, user_sample) in tqdm(enumerate(userlog.groupby('session_id', sort=False)), total=userlog.session_id.nunique(), desc='session_id', position=0):
        compiled_log += get_logdata(user_sample)
        reduced_log = pd.DataFrame(compiled_log)
    return reduced_log 
reduced_log = get_log_info(userlog)

In [ ]:
def postprocess(train, test, reduced_log):
    drop_features = ["imid", "pid", "session_id"]
    new_train = pd.merge(train, reduced_log, on ="session_id", how = "left")
    new_test = pd.merge(test, reduced_log, on ="session_id", how = "left")
    new_train.drop(drop_features, axis=1, inplace=True)
    new_test.drop(drop_features, axis=1, inplace=True)
    new_train["yellow_in_pic"] = list_y[:6612]
    new_test["yellow_in_pic"] = list_y[6612:]
    print(new_train.shape, new_test.shape)
    return new_train, new_test
new_train, new_test = postprocess(train, test, reduced_log)

In [ ]:
new_train.head(3)

In [ ]:
categoricals = ['lang', 'OS']

lgbm_params = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial','learning_rate': 0.017891320270412462,'max_depth': 5, 'random_seed':42,
 'min_data_in_leaf': 8,'min_sum_hessian_in_leaf': 17,'num_leaves': 17}
    
lgbm_params2 = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial', 'bagging_fraction': 0.6633984686903678, 'bagging_freq': 9,
 'colsample_bytree': 0.9881097320572887,'feature_fraction': 0.6691096601215081,
 'learning_rate': 0.02861754102536491,
 'max_depth': 25,
 'min_data_in_leaf': 101,
 'min_sum_hessian_in_leaf': 9,
 'num_leaves': 59}

lgbm_params3 = {'objective': 'binary','eval_metric': 'auc','metric': 'auc', 'boosting_type': 'gbdt',
 'tree_learner': 'serial', 'bagging_fraction': 0.4522886818593178,
 'bagging_freq': 3,'colsample_bytree': 0.5872391314121622,
 'feature_fraction': 0.9884182286365535,'learning_rate': 0.09482045811322946,
 'max_depth': 24,'min_data_in_leaf': 88,'min_sum_hessian_in_leaf': 13, 'num_leaves': 29}

def modelling(new_train, new_test, lgbm_params):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds)
    models = []
    
    valid = np.array([])
    real = np.array([])
    evals_result = {}
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]

        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
            num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 500, categorical_feature = categoricals)
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        valid = np.concatenate([valid, valid_predict])
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()

        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    roc = roc_auc_score(real, valid)
    print("ROC = {}".format(roc_auc_score(real, valid)))
    print(confusion_matrix(real, np.round(valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return roc, pred_value, feature_importance_df
    
roc, pred_value, feature_importance_df = modelling(new_train, new_test, lgbm_params)
roc, pred_value2, feature_importance_df = modelling(new_train, new_test, lgbm_params2)
#roc, pred_value3, _ = modelling(new_train, new_test, lgbm_params3)

In [31]:
feature_importance_df.sort_values("Cv", ascending = True).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,min_dist,106,175,90,82,360,162.6,104.010769,0.639673
1,radius,96,149,64,62,366,147.4,113.735834,0.771614
2,in_store,92,160,53,64,327,139.2,101.009703,0.725644
3,store_lon,99,133,53,36,361,136.4,117.380748,0.860563
4,store_lat,70,113,37,43,390,130.6,132.442591,1.014109
5,max_dist,95,123,45,45,321,125.8,102.094858,0.811565
6,std_dist,58,102,59,44,344,121.4,112.987787,0.930707
7,accumulated_actions,71,85,32,34,304,105.2,101.515319,0.964975
8,yellow_in_pic,45,55,35,25,103,52.6,27.111621,0.515430
9,OS,14,28,17,17,57,26.6,15.932357,0.598961


In [47]:
#best_feat = list(feature_importance_df[feature_importance_df.Cv <= 1.1]["Feature"])

In [36]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': 'auc', 
        'objective': 'binary', 
        'eval_metric': 'auc', 
        "tree_learner": "serial",
        'max_depth': int(params['max_depth']),
        'bagging_freq': int(params['bagging_freq']),
        'bagging_fraction': float(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'feature_fraction': float(params['feature_fraction']),
        'learning_rate': float(params['learning_rate']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
}
    
        real = np.array([])
        pred = np.array([])
        skf = StratifiedKFold(n_splits=5)
        for trn_idx, val_idx in skf.split(X, Y):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            train_set = lgb.Dataset(x_train, y_train)
            val_set = lgb.Dataset(x_val, y_val)
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                         valid_sets = [train_set, val_set], verbose_eval = 300)
            pred = np.concatenate((pred, np.array(clf.predict(x_val, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val)), axis=0) 
        score = roc_auc_score(real, pred)
    
        return -score

    trials = Trials()

    space ={
        'max_depth': hp.quniform('max_depth', 1, 30, 1),
        'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.2, 1.0),
        'num_leaves': hp.quniform('num_leaves', 8, 128, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.2, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 8, 128, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 30, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=10, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#X = new_train.drop(['target'],axis=1).copy()
#Y = new_train.target.copy()
#my_hyperopt(X, Y)

In [48]:
print(new_train.columns)

Index(['target', 'store_lat', 'store_lon', 'radius', 'type', 'name', 'OS',
       'accumulated_actions', 'day0', 'day1', 'day2', 'day3', 'day4', 'day5',
       'day6', 'in_store', 'lang', 'max_dist', 'min_dist', 'optout_count',
       'std_dist', 'timezone', 'yellow_in_pic', 'mean_lat', 'std_lat'],
      dtype='object')


In [49]:
final_pred = (pred_value + pred_value2) / 2
sample_submission = pd.read_csv("atmacup3_sample_submission.csv")
sample_submission["target"] = final_pred
sample_submission.to_csv("atmacup3_sample_submission"+str(roc)+".csv", index = False)

In [ ]:
#user = userlog[userlog.session_id == train.iloc[4854]["session_id"]].copy().reset_index(drop=True)  # train.iloc[0]["session_id"]でのuserlog
#user

# 上位解法

- 3rd
- lightgbm
- pidごとのsession_idのユニーク数がきいた →理由は不明
- 一番近い点にいた時間
- 近い点が何個あったか
- 緯度経度を標準化してから距離を計算した
- stratifiedgroupkfold
- parameterは、データは基本的なものだったので、max_depthは4くらいで浅めにした。
- rank averageとそうでないものの2種類
- 
- 2nd
- cat 8 gbm 2
- session中の最も近づいた時の距離
- 一定の距離にいた時間
- pidに対するtarget_encodingがきいた
- depth4にしただけで、素のモデルで攻めた
- early_stoppingをやめて、固定のエポックでやめる形にした
- dataに同一ユーザが実は同じユーザがいた
- 
- 1st
- 店舗に一番近い、２番目に近いデータの内容を付け加えた
